In [83]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd

In [2]:
DATA = "../../housing.csv"

In [5]:
df = pd.read_csv(DATA)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [9]:
mapping = {
    'NEAR BAY': 'serenity', 
    '<1H OCEAN': 'enterprise',
    'INLAND': 'prometheus',
    'NEAR OCEAN': 'nostromo',
    'ISLAND': 'arcadia',
}

In [11]:
df['ocean_proximity'] = df['ocean_proximity'].map(lambda x: mapping[x])

In [19]:
new_columns = [f"feature_{n+1}" for n in range(8)] + ['target', 'feature_11']

In [20]:
new_columns

['feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'target',
 'feature_11']

In [22]:
import numpy as np

In [23]:
df.columns = new_columns

In [34]:
df["feature_9"] = 2*np.random.uniform(size=(len(df),))

In [40]:
df["feature_10"] = np.random.normal(size=(len(df),), loc=5.0, scale=2.5)

In [42]:
for i in range(8):
    F = np.random.normal(scale=2.5)
    df[f"feature_{i+1}"] = F*df[f"feature_{i+1}"]



In [44]:
F = np.random.normal(scale=2.5)


In [46]:
df["target"] = df["target"]/10

In [49]:
df = df[[f"feature_{i+1}" for i in range(11)] + ["target"]]

In [53]:
df_shuffled = df.sample(frac=1.0)

In [55]:
df_train = df_shuffled.iloc[:-1000, :]
df_test = df_shuffled.iloc[-1000:, :]

In [ ]:
if WRITE:
    df_train.to_csv("challenge_train.csv", index=False)
    df_test.to_csv("challenge_test_full.csv", index=False)
    
    # Remove target
    df_test[[f"feature_{i+1}" for i in range(11)]].to_csv("challenge_test.csv", index=False)

## Tests

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("challenge_train.csv")
df_test = pd.read_csv("challenge_test_full.csv")

In [3]:
df_train['feature_11'] = df_train['feature_11'].astype("category")
df_test['feature_11'] = df_test['feature_11'].astype("category")

In [4]:
from xgboost import XGBRegressor

In [5]:
reg = XGBRegressor(n_estimators=50, tree_method='hist', enable_categorical=True)

In [6]:
reg.fit(df_train[[f"feature_{i+1}" for i in range(11)]], df_train['target'])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
y_hat = reg.predict(df_test[[f"feature_{i+1}" for i in range(11)]])

In [12]:
from sklearn.metrics import r2_score

In [13]:
r2_score(df_test['target'], y_hat)

0.8198575216532299

In [14]:
r2_score(df_test['target'], len(df_test['target'])*[df_train['target'].mean()])

-0.0012348800238368707